In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

import pandas as pd
import numpy as np
import csv


from sklearn.utils import shuffle

import pandas as pd
from PIL import Image
import numpy as np
import os
import csv
import re
from sklearn.model_selection import StratifiedKFold

In [2]:
np.random.seed(1024)
pos_data = pd.read_csv(r'sentence_embedding_list.csv', header=None)
neg_data = pd.read_csv(r'sentence_embedding_list_negative.csv', header=None)
com_data = pos_data.append(neg_data)
raw_data = shuffle(com_data)
raw_data.to_csv('suffle.csv', encoding = 'utf-8', header = None, index = False)
labels_dataset = raw_data[0]
labels_dataset = labels_dataset.tolist()
raw_data

,0,1
25,1,[-2.89027214e-01 -4.38273013e-01 -3.22299331e-...
89,0,[-7.12146819e-01 -4.24583733e-01 -1.29917473e-...
597,1,[-2.12954760e-01 -8.21731985e-02 -2.26242959e-...
153,1,[-5.63706942e-02 -3.41074973e-01 -1.33537129e-...
711,0,[-4.71603841e-01 -6.57357454e-01 1.23730779e-...
...,...,...
601,1,[ 1.25624493e-01 -1.17772870e-01 2.84849796e-...
613,1,[-2.85503119e-01 -2.00217754e-01 -7.87503868e-...
492,1,[-1.12773709e-01 -4.28319186e-01 -1.97886527e-...
609,1,[-1.75341040e-01 8.72757733e-02 -1.16240516e-...


In [3]:
raw = pd.read_csv(r'suffle.csv', header=None)
dataset = []
for data in raw[1].tolist() :
    num = re.findall('[0-9e.-0-9+]+',data)
    np_num = np.array([float(i) for i in num])
    dataset.append(np_num) 

In [4]:
batch_size = 32  #一個撮（批次）的大小

In [5]:
# 首先，我們定義索引陣列indices，它相當於對所有dataset中數據的編碼
# 然後定義索引indices_train表示測試級的索引，indices_val來表示校驗集數據的索引，indices_test表示測試集的索引
indices = range(len(dataset))
data_size = len(dataset) // 10
indices_train = indices[2 * data_size :]
indices_val = indices[: data_size]
indices_test = indices[data_size : 2 * data_size]

# 根據這些索引，構造三個數據集的SubsetRandomSampler採樣器，它會對索引進行採樣
sampler_train = torch.utils.data.sampler.SubsetRandomSampler(indices_train)
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(indices_val)
sampler_test = torch.utils.data.sampler.SubsetRandomSampler(indices_test)

# 根據三個採樣器來定義加載器，注意將sampler_train、sampler_val和sampler_test分別賦值給了train_loader、validation_loader和test_loader
#注意:labels集只需用相同定義的sampler就可以採樣

train_loader_train = torch.utils.data.DataLoader(dataset= dataset, 
                                           batch_size= batch_size, 
                                           sampler = sampler_train)
train_loader_labels = torch.utils.data.DataLoader(dataset= labels_dataset, 
                                           batch_size= batch_size, 
                                           sampler = sampler_train)

validation_loader_test = torch.utils.data.DataLoader(dataset = dataset,
                                                batch_size = batch_size,
                                                sampler = sampler_val)
validation_loader_labels = torch.utils.data.DataLoader(dataset =labels_dataset,
                                                batch_size = batch_size,
                                                sampler = sampler_val)

test_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=batch_size, 
                                          sampler = sampler_test)
test_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=batch_size, 
                                          sampler = sampler_test)




In [7]:
len(sampler_train)

1442

In [6]:
num_classes = 2  #標簽的種類數
num_epochs = 100  #訓練的總循環周期

size = 768
hidden1 = 100
hidden2 = 100
out = 2

In [8]:
class NN(nn.Module):
    def __init__(self, size, hidden1, hidden2, num_classes):
        # 該函數在創建一個NN對象的時候，即調用如下語句：model=NN()，就會被調用
        # 首先調用父類相應的構造函數
        super(NN, self).__init__()
        
        # 其次構造NN需要用到的各個神經模塊。
        '''注意，定義組件並沒有真正搭建這些組件，只是把基本建築磚塊先找好'''
        
        # 全連線層
        self.fc1 = nn.Linear(size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, num_classes) 
        
        #BN
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.bn2 = nn.BatchNorm1d(hidden2)
        
        
        
        
        

    def forward(self, x):
        
        x = self.fc1(x)
        x = F.relu(self.bn1(x))
        x = F.dropout(x, training  = self.training)
        
        x = self.fc2(x)
        x = F.relu(self.bn2(x))
        x = F.dropout(x, training  = self.training)
        
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)#輸出層為log_softmax，即概率對數值log(p(x))。采用log_softmax可以使得後面的交叉熵計算更快

        return x

In [9]:
model = NN(size, hidden1, hidden2, num_classes)
print(model)

NN(
  (fc1): Linear(in_features=768, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=2, bias=True)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [10]:
params = list(model.parameters())
print('引數: ', params)
# conv1.weight
print('第一個引數大小: ', params[-1].size())

引數:  [Parameter containing:
tensor([[-0.0155, -0.0040,  0.0133,  ..., -0.0107, -0.0243, -0.0225],
        [-0.0117,  0.0182, -0.0018,  ..., -0.0293,  0.0232, -0.0172],
        [-0.0344,  0.0313, -0.0194,  ..., -0.0020,  0.0220, -0.0322],
        ...,
        [-0.0261, -0.0081, -0.0140,  ...,  0.0080,  0.0263, -0.0232],
        [ 0.0252,  0.0156,  0.0067,  ...,  0.0114,  0.0248, -0.0108],
        [-0.0112, -0.0182, -0.0244,  ...,  0.0333,  0.0208, -0.0037]],
       requires_grad=True), Parameter containing:
tensor([-0.0069, -0.0108, -0.0270, -0.0338, -0.0323,  0.0059,  0.0319,  0.0064,
         0.0259, -0.0031,  0.0068,  0.0319, -0.0304, -0.0189, -0.0337,  0.0025,
         0.0087,  0.0280,  0.0201, -0.0244, -0.0131, -0.0186,  0.0247, -0.0306,
        -0.0167,  0.0247, -0.0056,  0.0135,  0.0356, -0.0340, -0.0261,  0.0312,
         0.0223, -0.0291, -0.0038, -0.0133,  0.0082,  0.0091, -0.0037, -0.0149,
        -0.0027, -0.0055,  0.0091, -0.0304, -0.0230,  0.0345,  0.0122,  0.0028,
        

In [11]:
model = NN(size, hidden1, hidden2, num_classes)

# 損失函數為交叉熵
criterion = nn.CrossEntropyLoss()
# 優化算法為Adam，可以自動調節學習率
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)



def rightness(predictions, labels):
    """计算预测错误率的函数，其中predictions是模型给出的一组预测结果，batch_size行num_classes列的矩阵，labels是数据之中的正确答案"""
    pred = torch.max(predictions.data, 1)[1] # 对于任意一行（一个样本）的输出值的第1个维度，求最大，得到每一行的最大元素的下标
    rights = pred.eq(labels.data.view_as(pred)).sum() #将下标与labels中包含的类别进行比较，并累计得到比较正确的数量
    return rights, len(labels) #返回正确的数量和这一次一共比较了多少元素

In [12]:
for i, data in enumerate(zip(train_loader_train, train_loader_labels)):
    x, y = data
    z = torch.tensor(x, requires_grad = True, dtype = torch.float) 
    
model(z)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


tensor([[-0.5468, -0.8646],
        [-1.2829, -0.3247]], grad_fn=<LogSoftmaxBackward>)

In [13]:
for i, data in enumerate(zip(train_loader_train, train_loader_labels)):
    x, y = data
    print(x, y)

tensor([[-0.2533,  0.1856,  0.3003,  ..., -0.2644,  0.3201,  0.5865],
        [-0.1071,  0.0285, -0.2822,  ...,  0.1530, -0.1541,  0.5235],
        [-0.3505, -0.1567,  0.0972,  ...,  0.5291,  0.2423,  0.2224],
        ...,
        [-0.9522, -0.4808, -0.6104,  ..., -0.4280,  0.4121, -0.1257],
        [-0.0512, -0.1326, -0.1711,  ..., -0.1748,  0.1237,  0.0227],
        [-0.0658, -0.0026, -0.2920,  ..., -0.5041,  0.4575, -0.0334]],
       dtype=torch.float64) tensor([1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 0])
tensor([[-0.2569, -0.2388, -0.2662,  ..., -0.3022,  0.0228,  0.8991],
        [-0.0481, -0.0033,  0.2663,  ...,  0.0056, -0.0894,  0.5069],
        [ 0.0183, -0.3482, -0.3320,  ...,  0.0327,  0.1526,  0.4150],
        ...,
        [-0.7733, -0.1829,  0.0613,  ..., -0.1642,  0.4151,  0.7047],
        [-0.2603,  0.1095,  0.0656,  ...,  0.0616, -0.0529,  0.5853],
        [-0.1469, -0.3186, -0.1479,  ...,  0.4047,  0.2063,  0

In [ ]:
import warnings
warnings.filterwarnings('ignore')
record = []
losses = []
records = []
for epoch in range(num_epochs):
    train_losses = []
    train_rights = []
    model.train()
    for batch_idx, train in enumerate(zip(train_loader_train, train_loader_labels)):
        x, y = train
        
        
        x = torch.tensor(x, requires_grad = True, dtype = torch.float)   # x 最後會被轉成 [0-1] 之間的數

        y = torch.tensor(np.array([z for z in y ]), dtype = torch.long)  # 所以 y 必須是llong 而不是
        
        #np.array([z for z in y ]) 因為np.array只能轉一個值，若要list裡面全部的值，需改成這樣
        
        # 清空梯度
        optimizer.zero_grad()
        # 模型預測
        predict = model(x)
        # 計算損失函數
        loss = criterion(predict, y)
        # 將損失函數數值加入到列表中
        train_losses.append(loss.data.numpy())
        # 開始進行梯度反傳
        loss.backward()
        # 開始對參數進行一步優化
        optimizer.step()
        #計算準確率所需數值，返回數值為（正確樣例數，總樣本數）
        right = rightness(predict, y)
        #將計算結果裝到列表容器train_rights中
        train_rights.append(right)
        
        
        
        
        
        # 每隔200步，跑一下校驗數據集的數據，輸出臨時結果
        if batch_idx % 10 == 0:
        
            losses.append(np.mean(train_losses))
            model.eval()
            val_losses = []
            val_rights = []
            # 在所有校驗數據集上實驗
            with torch.no_grad():
                
                for j, val in enumerate(zip(validation_loader_test, validation_loader_labels)):
                    x, y = val
                    x = torch.tensor(x, requires_grad = True, dtype = torch.float)
                    y = torch.tensor(np.array([z for z in y ]), dtype = torch.long)
                    predict = model(x)
                    # 調用rightness函數計算準確度
                    right = rightness(predict, y)
                    val_rights.append(right)
                    loss = criterion(predict, y)
                    val_losses.append(loss.data.numpy())



                # 分別計算在目前已經計算過的測試數據集，以及全部校驗集上模型的表現：分類準確率
                #train_r為一個二元組，分別記錄目前已經經歷過的所有訓練集中分類正確的數量和該集合中總的樣本數，
                #train_r[0]/train_r[1]就是訓練集的分類準確度，同樣，val_r[0]/val_r[1]就是校驗集上的分類準確度
                train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
                #val_r為一個二元組，分別記錄校驗集中分類正確的數量和該集合中總的樣本數
                val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
                #打印準確率等數值，其中正確率為本訓練周期Epoch開始後到目前撮的正確率的平均值
                print(val_r)
                print('訓練周期: {} [{}/{} ({:.0f}%)]\tTotal Loss: {:.2f}\ttraining Loss: {:.2f}\tval Loss: {:.2f}\t訓練正確率: {:.2f}%\t校驗正確率: {:.2f}%'.format(
                    epoch, batch_idx * batch_size, len(dataset),
                    100. * batch_idx / len(train_loader_train), 
                    np.mean(losses),
                    np.mean(train_losses),
                    np.mean(val_losses),
                    100. * train_r[0].numpy() / train_r[1], 
                    100. * val_r[0].numpy() / val_r[1]))
                #將準確率和權重等數值加載到容器中，以方便後續處理
                record.append((100 - 100. * train_r[0] / train_r[1], 100 - 100. * val_r[0] / val_r[1]))
                records.append([np.mean(train_losses), np.mean(val_losses), train_r, val_r, losses])

(tensor(92), 180)
訓練周期: 0 [0/1802 (0%)]	Total Loss: 0.74	training Loss: 0.74	val Loss: 0.69	訓練正確率: 53.12%	校驗正確率: 51.11%
(tensor(86), 180)
訓練周期: 0 [320/1802 (22%)]	Total Loss: 0.72	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.58%	校驗正確率: 47.78%
(tensor(87), 180)
訓練周期: 0 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.51%	校驗正確率: 48.33%
(tensor(88), 180)
訓練周期: 0 [960/1802 (65%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 48.59%	校驗正確率: 48.89%
(tensor(86), 180)
訓練周期: 0 [1280/1802 (87%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.16%	校驗正確率: 47.78%
(tensor(82), 180)
訓練周期: 1 [0/1802 (0%)]	Total Loss: 0.72	training Loss: 0.78	val Loss: 0.70	訓練正確率: 40.62%	校驗正確率: 45.56%
(tensor(85), 180)
訓練周期: 1 [320/1802 (22%)]	Total Loss: 0.72	training Loss: 0.70	val Loss: 0.69	訓練正確率: 49.43%	校驗正確率: 47.22%
(tensor(90), 180)
訓練周期: 1 [640/1802 (43%)]	Total Loss: 0.71	training Loss: 0.70	val Loss: 0.70	訓練正確率: 51.79%	校驗正確率: 50.00%
(tensor(92), 180)
訓練周

#### label 2名稱

In [ ]:
#繪制訓練過程的誤差曲線，校驗集和測試集上的錯誤率。
plt.figure(figsize = (10, 7))
plt.plot(record, label = 'train_acc') #record記載了每一個打印周期記錄的訓練和校驗數據集上的準確度
plt.xlabel('Steps')
plt.ylabel('Error rate')
plt.legend()
plt.ylim(0, 1)

In [ ]:
records.append([np.mean(train_losses), np.mean(val_losses), train_r, val_r, losses])

In [ ]:
# 繪制誤差曲線
a = [i[0] for i in records]
b = [i[1] for i in records]
c = [i[2] for i in records]
d = [i[3] for i in records]
e = [i[4] for i in records]
plt.plot(a, label = 'Train Loss')
plt.plot(b, label = 'Valid Loss')
plt.plot(c, label = 'Train Accuracy')
plt.plot(d, label = 'Valid Accuracy')
plt.xlabel('Steps')
plt.ylabel('Loss & Accuracy')
plt.legend()

In [ ]:
torch.save(model,'bow.mdl')
model = torch.load('bow.mdl')

In [ ]:
#在測試集上分批運行，並計算總的正確率
vals = [] #記錄準確率所用列表

#對測試數據集進行循環
for data, target in zip(test_data, test_label):
    data, target = torch.tensor(data, dtype = torch.float).view(1,-1), torch.tensor(np.array([target]), dtype = torch.long)
    output = model(data) #將特征數據喂入網絡，得到分類的輸出
    val = rightness(output, target) #獲得正確樣本數以及總樣本數
    vals.append(val) #記錄結果

#計算準確率
rights = (sum([tup[0] for tup in vals]), sum([tup[1] for tup in vals]))
right_rate = 1.0 * rights[0].data.numpy() / rights[1]
right_rate